In [ ]:
import random
import ast
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import axelrod as axl




In [ ]:
from joblib import Parallel, delayed
import xgboost as xgb


In [ ]:
strategies = [axl.FirstByDavis(),axl.ContriteTitForTat(), axl.OnceBitten(), axl.InversePunisher(), axl.InversePunisher()]


In [ ]:
strategy_order = [player.__class__.__name__ for player in strategies]

strategy_mapping = {name: player.__class__ for name, player in zip(strategy_order, strategies)}

strategy_mapping

In [ ]:
MAX_COUNT = 5          # each strategy appears between 0 and 5 times
NUM_TOTAL_COMPS = (MAX_COUNT+1) ** len(strategy_order)  # 6^5 = 7776
NUM_SIMULATIONS = 5000   # number of tournaments to simulate (sampled from all compositions)
TOURNS = 200           # turns per tournament
REPETITIONS = 10      # repetitions per tournament


def get_tournament_winners_by_score(results, players, use_normalised=True):
    if use_normalised:
        if hasattr(results.normalised_scores, "compute"):
            scores_matrix = np.array(results.normalised_scores.compute())
        else:
            scores_matrix = np.array(results.normalised_scores)
    else:
        if hasattr(results.scores, "compute"):
            scores_matrix = np.array(results.scores.compute())
        else:
            scores_matrix = np.array(results.scores)
    average_scores = [np.mean(scores) for scores in scores_matrix]
    score_dict = {player.__class__.__name__: avg for player, avg in zip(players, average_scores)}
    print("Average scores:", score_dict)
    max_score = max(average_scores)
    winners = [
        player.__class__.__name__ for player, avg in zip(players, average_scores)
        if abs(avg - max_score) < 0.001
    ]
    unique_winners = []
    for w in winners:
        if w not in unique_winners:
            unique_winners.append(w)
    print("Winners for this tournament:", unique_winners, "\n")
    return unique_winners

def run_single_tournament(composition, turns=TOURNS, repetitions=REPETITIONS):
    players = []
    for count, strat_name in zip(composition, strategy_order):
        for _ in range(count):
            players.append(strategy_mapping[strat_name]())
    print("Tournament composition:", composition)
    tourn = axl.Tournament(players, turns=turns, repetitions=repetitions)
    results_tourn = tourn.play()
    coop_matrix = results_tourn.cooperation
    total_cooperations = sum(sum(row) for row in coop_matrix)
    total_moves = len(coop_matrix) ** 2 * repetitions
    coop_rate = total_cooperations / total_moves if total_moves > 0 else 0.0
    winners = get_tournament_winners_by_score(results_tourn, players)
    return {
        "composition": composition,
        "coop_rate": coop_rate,
        "winning_strategy": winners
    }



In [ ]:
all_comps = [tuple(x) for x in itertools.product(range(MAX_COUNT + 1), repeat=len(strategy_order))]
all_comps = [x for x in all_comps if any(x)]

print("Total possible compositions:", len(all_comps))  

selected_comps = random.sample(all_comps, NUM_SIMULATIONS)
print("Selected compositions for simulation:", len(selected_comps))


def run_single_tournament_wrapper(comp, idx):
    print(f"Starting tournament simulation {idx+1} of {NUM_SIMULATIONS}")
    return run_single_tournament(list(comp), turns=TOURNS, repetitions=REPETITIONS)

# Run the simulations in parallel.
from joblib import Parallel, delayed
sim_results = Parallel(n_jobs=-1, verbose=10)(
    delayed(run_single_tournament_wrapper)(comp, idx) for idx, comp in enumerate(selected_comps)
)

In [ ]:
df_baseline = pd.DataFrame(sim_results)


In [ ]:
df_baseline.to_csv('../simulated_data/simulated_tournamets/gd_baseline.csv')

In [ ]:
# Generalized Defector


class GeneralizedDefector(axl.Player):
    name = "Generalized Defector"
    classifier = {
        "memory_depth": float("inf"),
        "stochastic": False,
        "makes_use_of": set(),
        "long_run_time": False,
        "inspects_source": False,
        "manipulates_source": False,
        "manipulates_state": False,
    }

    def __init__(self, n=1, m=1):
        super().__init__()
        self.n = n
        self.m = m

    def strategy(self, opponent: axl.Player) -> axl.Action:
        round_number = len(self.history) + 1
        if round_number < self.m:
            return axl.Action.C

        elif (round_number - self.m) % self.n == 0:
            return axl.Action.D
        else:
            return axl.Action.C

In [ ]:
S = len(strategy_order)            # 5
MAX_COUNT  = 5
MAX_GD     = 5
MAX_ITER   = 5000
TOURNS     = 200
REPET      = 1


core_mapping = {
    "FirstByDavis": axl.FirstByDavis(),              
    "ContriteTitForTat": axl.ContriteTitForTat(),
    "OnceBitten": axl.OnceBitten(),
    "InversePunisher":  axl.InversePunisher(), 
    "EvolvedANNNoise05": axl.InversePunisher(), 
}



In [ ]:
import random as rng
def random_base_counts():
    local_rng = random.Random() 
    return [local_rng.randint(0, MAX_COUNT) for _ in range(S)]

def gd_parameters(iter_idx):
    cyc = (iter_idx % 5) + 1
    return cyc, cyc

def make_composition(iter_idx):
    """
    Improved version:
    - Random base counts
    - Random slot (not fixed)
    - Random k for GD (1..MAX_GD) or maybe no GD at all (small probability)
    """
    counts = random_base_counts()
    slot = rng.randint(0, S-1)         

    if rng.random() < 0.9:
        k = rng.randint(1, MAX_GD)       
        counts[slot] = max(k, counts[slot]) 
    else:
        k = 0 

    n, m = gd_parameters(iter_idx)
    return counts, slot, k, n, m

def play_tournament(counts, gd_slot, gd_k, gd_n, gd_m, core_mapping,
                    turns=TOURNS, repetitions=REPET):
    players = []
    for idx, (cnt, name) in enumerate(zip(counts, strategy_order)):
        cls = GeneralizedDefector if idx == gd_slot else core_mapping[name].__class__
        for _ in range(cnt):
            if idx == gd_slot:
                players.append(cls(n=gd_n, m=gd_m))
            else:
                players.append(cls())

    tour = axl.Tournament(players, turns=turns, repetitions=repetitions)
    res  = tour.play()

    coop_rt = (np.sum(res.cooperation) /
               (len(players)**2 * repetitions)) if players else 0.0

    avg_sc  = np.mean(np.array(res.normalised_scores), axis=1)
    winners = [pl.__class__.__name__
               for pl, sc in zip(players, avg_sc)
               if abs(sc-avg_sc.max()) < 1e-3]
    winners = list(dict.fromkeys(winners))

    return dict(
        composition=counts,
        gd_slot=gd_slot, gd_k=gd_k, gd_n=gd_n, gd_m=gd_m,
        coop_rate=coop_rt,
        winning_strategy=winners,
    )

In [ ]:
def wrapper(iter_idx, core_mapping):
    counts, slot, k, n, m = make_composition(iter_idx)
    print(f"sim {iter_idx+1:>4d}/{MAX_ITER}  slot={slot}  k={k}  (n,m)=({n},{m})")
    return play_tournament(counts, slot, k, n, m, core_mapping)

results = Parallel(n_jobs=-1, verbose=10)(
    delayed(wrapper)(i, core_mapping) for i in range(MAX_ITER)
)

df_GD = pd.DataFrame(results)

print("\nSample:")
display(df_GD.head())

In [ ]:
df_GD.to_csv('../simulated_data/simulated_tournamets/strong_plus_GD.csv')